Purpose: clean and aggregate multi-tab leader tracker into one sheet
- link https://docs.google.com/spreadsheets/d/186JSFYCzweoyct0RtCZcYwp6nSFZb22zYBHVYHqKTPA/edit#gid=0

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 75)

import matplotlib.pyplot as plt

# TO DO:
 - Put Leader tracker in a form to be imported into EA regularly. (main priority)
     - Clean up names (done)
     - Standardize leadership status (done)
     - Clean up Phone numbers (done)
     - Clean up zip codes (done)
     - Remove duplicates
     - Assign Organization, Membership (done)
     - Identify institutions that may need to be cleaned before initial upload (bolded in csv)
 - Need counts and analyses that are exportable (later, nobody has asked for this lol)
     - How many (overall, by organizer, by region?, by institution?, by leadership status?, by zip?, by recruited by?, title): people, phones, emails, zips, institutions, leadership status, core team, ROL, recruited by
     - Viz: bar chart - #ppl per organizer, 
     
# Questions
 - Removing duplicates by email, first, and last. Or email and last (see who these ppl are)?
 - How to make df.drop_duplicates() not take NaN into account? (Tianyi)
      - try: df[(~df.duplicated()) | (df['col'].isnull())] according to the internet?
      - include organizer in subset?
 - When I upload to EA want to assign organizer to people I think. What if a person is on multiple organizers lists? See if more than one person can be assigned to a person in EA?
 - Column counts by recruited by, in order of how many names they have
     - Daniel has a lot of records with only emails and no names
 - Can switch leadership status mappings to be the ones without numbers
 - Does Edwin have a list in our AN setup? is he in BFO?

In [2]:
# Read all tabs you want from leader tracker and write in df
dataset = r'C:\Users\OOC User\Documents\OOC\Data\Leader Tracking\2022\OOC 2022 Leaders.xlsx'
data = pd.ExcelFile(dataset) 
df1 = pd.DataFrame() #where the imported data will go after formatting


#read in data
for i in range(5, len(data.sheet_names)): #don't need first 2 or last tabs in file
    df_temp = pd.read_excel (dataset, sheet_name = data.sheet_names[i], header = 3, dtype = {'Phone':str})#Read an organizers sheet
    df_temp = df_temp.loc[:,:'Notes & Reflections'] #Take first column through notes column only
    df_temp['Organizer'] = data.sheet_names[i]#add column for organizer
    df_temp = df_temp.loc[~(df_temp['First Name'].isnull() & df_temp['Last Name'].isnull() & df_temp['E-Mail'].isnull()),:] #Remove rows with null first/last name/Email
    df_temp = df_temp.iloc[:-1] #Remove last row
    df1 = pd.concat([df1, df_temp]) #add df_temp todf1

# #strip everything? this didnt work lol
# for x in df1:
#     df1[x] = df1[x].astype(str).str.strip()
# df1.fillna(value='nan', inplace=True)
    
    
#drop index
df1 = df1.reset_index(drop=True)

# Core Team, Relational Organizing Leader Mapping

~~~
0 > No
1 > Yes
~~~


In [3]:
df1.loc[:, 'Relational Organizing Leader'] = df1.loc[:, 'Relational Organizing Leader'].map({0.0:'No', 1.0:'Yes'})
df1.loc[:, 'Core Team'] = df1.loc[:, 'Core Team'].map({0.0:'No', 1.0:'Yes'})

# Leadership Status Mapping

~~~
'Prospect': '1-Prospect'
'Volunteer Leader': '2-Volunteer'
'Member':'3-Member Leader
'Super Leader':'4-Super Leader'
'0-N/A': BLANK
~~~

In [4]:
#Leadership status mapping
df1.loc[df1['Leadership Status']=='Prospect', 'Leadership Status'] = '1-Prospect'
df1.loc[df1['Leadership Status']=='Volunteer Leader', 'Leadership Status'] = '2-Volunteer'
df1.loc[df1['Leadership Status']=='Member', 'Leadership Status'] = '3-Member Leader'
df1.loc[df1['Leadership Status']=='Super Leader', 'Leadership Status'] = '4-Super Leader'
df1.loc[df1['Leadership Status']=='0-N/A', 'Leadership Status'] = np.nan

# Phone Numbers
- Remove non-digit characters
- Output numbers for follow-up if:
    - number length not 7, 10, or 11
    - number length is 11 but doesn't start with U.S. country code 1

In [5]:
# Checking phone numbers
#Remove all non digits characters
df1.loc[:, 'Phone'] = df1.loc[:, 'Phone'].str.replace("\D", '', regex=True) #remove nondigit characters

In [6]:
df1.loc[:, 'Phone'].value_counts()

93735077630    3
73267226580    3
30127589430    3
93723283130    3
4046210311     2
              ..
61435217230    1
93751227420    1
93747527660    1
9373507763     1
5132379703     1
Name: Phone, Length: 397, dtype: int64

In [7]:
# Names with non-letter characters
# Output first/last names that contain characters that aren't letters
first_new = df1['First Name'].str.strip().str.contains("[^a-z]", case=False, regex=True)
last_new = df1['Last Name'].str.strip().str.contains("[^a-z]", case=False, regex=True)
names_new = df1.loc[first_new | last_new, ['First Name', 'Last Name', 'E-Mail', 'Phone', 'Zip Code', 'Organizer']]
names_new = names_new.reset_index(drop=True)

# #read in previous names
# # prev_date = '7.25.2021' #should be defined above
# names_old = pd.read_csv(f'Old_Names_{prev_date}.csv')
# # names_old = pd.read_csv('Old_Names_7.25.2021.csv')
# names_old = names_old.reset_index(drop=True)

# common = pd.merge(left=names_new, right=names_old, on=['First Name','Last Name', 'Organizer'], how='outer', indicator=True)
# new_only = common.loc[common['_merge']=='left_only',:]
# old_only = common.loc[common['_merge']=='right_only',:]

# #Are there old names missing from new names list?
# if len(old_only) > 0:
#     print("WARNING: Old names are missing from new names list\nWARNING: Investigate \"Old_names_to_check.csv\"\n\n")
# #     old_only.to_csv('Old_names_to_check.csv', index=False)
    
#     #print Old Names to check
#     print('Old names to check\n')
#     display(old_only)
# else:
#     print("All old names were found on the new names list, no old names to check\n\n")

    
# #Are there any new names missing from the old names list?
# if len(new_only) > 0:
#     print("WARNING: New names that aren't on old list\nWARNING: Investigate \"New_names_to_check.csv\"\n\n")
# #     new_only.to_csv('New_names_to_check.csv', index=False)
    
#     #print Old Names to check
#     print('New names to check\n\n')
#     display(new_only)
    
# else:
#     print('All new names found in old names list, no new names to check\n\n')

# #output all new names to be checked against for next time
# names_new.to_csv(f'Old_Names_{date}.csv', index=False)
# print(f'Prev. Date for next time: {date}\n\n')

# Organization and Membership Mapping

    Caring Economy (name TBD)  - none
        • Child Care - Tami
        • Unemployment - Kalia
    Amos Project - none
        • Amos Cinci - Jacob, Daniel
        • Amos Columbus - Derrick H.
        • Amos Cleveland - none
    Ohio Student Association  - none
        • OSA Cleveland -  Rachael
        • OSA Columbus - Akii
        • OSA Dayton - Kalesha
        • OSA Cincinnati - none, chech with Rachael if she has a cincy list
    Building Freedom Ohio  - none
        • BFO Cleveland - Robin
        • BFO Cinci - Celeste
        - BFO Dayton - Edwin

In [8]:
#Assigning Organization

# WHAT TO DO WHEN ORGANIZERS CHANGE? IE JANIAH HAS AN ASSIGNED ORG, WHAT HAPPENS WHEN SHE LEAVES OOC? 
# WHAT MEMBER SHOULD JANIAH BE? CINCINNATI?

#Caring Economy
df1.loc[(df1['Organizer'].str.contains("Tami|Janiah")), 'Organization'] = 'Caring Economy'
#AMOS
df1.loc[(df1['Organizer'].str.contains("Daniel|Derrick")), 'Organization'] = 'AMOS'
#OSA
df1.loc[(df1['Organizer'].str.contains("Rachael|Akii|Kalesha")), 'Organization'] = 'OSA'
#BFO
df1.loc[(df1['Organizer'].str.contains("Edwin")), 'Organization'] = 'BFO'   

#Assigning Membership
#Caring Economy
df1.loc[df1['Organizer']=="Tami", 'Membership'] = 'CE: Child Care'
df1.loc[df1['Organizer']=="Janiah", 'Membership'] = 'CE: Unemployment'
#AMOS
df1.loc[df1['Organizer']=="Daniel", 'Membership'] = 'AMOS: Cincinnati'
df1.loc[df1['Organizer']=="Derrick", 'Membership'] = 'AMOS: Columbus'
#OSA
df1.loc[df1['Organizer']=="Rachael", 'Membership'] = 'OSA: Cleveland'
# df1.loc[df1['Organizer']=="Rachael1", 'Membership'] = 'OSA: Cleveland' #Ask about this first
df1.loc[df1['Organizer']=="Akii", 'Membership'] = 'OSA: Columbus'
df1.loc[df1['Organizer']=="Kalesha", 'Membership'] = 'OSA: Dayton'
#BFO
df1.loc[df1['Organizer']=="Robin", 'Membership'] = 'BFO: Cleveland'
df1.loc[df1['Organizer']=="Edwin", 'Membership'] = 'BFO: Dayton'

In [9]:
df1.Organizer.value_counts()

Edwin       400
Rachael     267
Rachael1    139
Daniel      125
Tami         56
Kalesha      43
Akii         42
Derrick      32
Janiah       21
Name: Organizer, dtype: int64

In [10]:
#Organizers without an organization/membership
#Assign organization? Move records to another list? 
no_org = df1.loc[df1['Organization'].isnull(), ['Organizer', 'Organization']]['Organizer'].value_counts()
no_org

Series([], Name: Organizer, dtype: int64)

# Title Mapping
~~~
Acceptable mappings and titles:
    Pastor
    Rev./Reverend >> Rev
    Bishop
    Dr. >> Dr
    Rabbi
    Evangelist?
    Rev Dr
~~~



In [11]:
acceptable_titles = ['Pastor', 'Rev', 'Bishop', 'Dr', 'Rabbi', 'Evangelist', 'Rev Dr']
df1['Title'] = df1['Title'].astype(str).str.strip()
df1.loc[df1['Title']=='nan','Title']=np.nan

#Title Mapping
df1.loc[df1['Title'].isin(["Rev.", "Reverend"]), ['Title']] = 'Rev'
df1.loc[df1['Title'].isin(["Dr."]), ['Title']] = 'Dr'

#New titles to consider?
titles_new = df1.loc[~(df1['Title'].isin(acceptable_titles)) & (df1['Title'].notnull()), :]
titles_new
if len(titles_new) > 0:
    print("The following TITLES have been output for follow-up, investigate 'Titles_to_check.csv': \n")
    display(titles_new)
    titles_new.to_csv(f'Titles_to_check.csv', index=False)
else:
    print("No new titles to consider")

No new titles to consider


# Zip Codes

In [12]:
# Output zip if nondigit character or first digit is not 4
# Check against zips from previous run

#convert to string
df1['Zip Code'] = df1['Zip Code'].astype(str).str.strip()
#first 5 characters
df1['Zip Code'] = df1['Zip Code'].str.slice(0,5)
# #Remove non digit characters
# non_digits = df1['Zip Code'].str.contains("[^0-9]", case=False, regex=True)
# not_na = ~df1['Zip Code'].str.contains('nan')
# non_digits = df1.loc[non_digits & not_na, 'Zip Code']
# # display(non_digits)
# #first digit is not 4
# not_4 = ~df1['Zip Code'].str.startswith('4', na=False)
# not_4 = df1.loc[not_4 & not_na, 'Zip Code']
# # display(not_4)
# #put it all together
# zips_new = df1.loc[df1['Zip Code'].isin(list(not_4)) | df1['Zip Code'].isin(list(non_digits)), ['First Name', 'Last Name', 'E-Mail', 'Phone', 'Zip Code', 'Organizer']]
# zips_new = zips_new.reset_index(drop=True)


# #read in previous zips
# # prev_date = '7.25.2021' #should be defined above already
# zips_old = pd.read_csv(f'old_zips_{prev_date}.csv')
# # names_old = pd.read_csv('Old_Names_7.25.2021.csv')
# zips_old = zips_old.reset_index(drop=True)

# common_zips = pd.merge(left=zips_new, right=zips_old, on=['First Name','Last Name', 'Zip Code', 'Organizer'], how='outer', indicator=True)
# new_only = common_zips.loc[common_zips['_merge']=='left_only',:]
# old_only = common_zips.loc[common_zips['_merge']=='right_only',:]

# #Are there old zips missing from new zips list?
# if len(old_only) > 0:
#     print("WARNING: Old zips are missing from new zips list, hopefully they were cleaned in the leader tracker? \"old_zips_to_check.csv\"")
# #     old_only.to_csv('old_zips_to_check.csv', index=False)
    
#     #print Old Names to check
#     print('Old zips to check:\n')
#     display(old_only)
# else:
#     print("All old zips were found on the new zips list, no old zips to check\n\n")
    
# #Are there any new zips missing from the old zips list?
# if len(new_only) > 0:
#     print("\n\nWARNING: New zips that aren't on old list, investigate if they're legit \"New_zips_to_check.csv\"")
# #     new_only.to_csv('New_zips_to_check.csv', index=False)
    
#     #print Old zips to check
#     print('New zips to check:\n\n')
#     display(new_only)
    
# else:
#         print('All new zips found in old names list, no new zips to check\n\n')
        
# #output all new zips to be checked against for next time
# zips_new.to_csv(f'Old_zips_{date}.csv', index=False)
# print(f'Prev. Date for next time: {date}\n\n')

In [13]:
#Remove @ohorganizing.org emails  
df = df1.loc[~df1['E-Mail'].str.contains('@ohorganizing.org', na=False), :] 

# Duplicates

In [14]:
#Drop exact duplicates

print(f'Started with {str(len(df))} records\n\n')

exact_dups = df[df.duplicated()]
email_name_dups = df[df.duplicated(subset = ['E-Mail', 'First Name', 'Last Name'])]
email_last_dups = df[df.duplicated(subset = ['E-Mail', 'Last Name'])]

#Remove exact duplicates
print(f'Exact duplicates removed: {len(exact_dups)}')
df = df.drop_duplicates()

#Remove duplicates with matching email/first/last
print(f'Email/First/Last duplicates removed: {len(email_name_dups)}')
df = df.drop_duplicates(subset = ['E-Mail', 'First Name', 'Last Name'])

# # Remove duplicates with matching email/last
# print(f'Email/Last duplicates removed: {len(email_last_dups)}\n\n')
# df = df.drop_duplicates(subset = ['E-Mail', 'Last Name'])

print(f'{len(df)} records remaining')

Started with 1120 records


Exact duplicates removed: 6
Email/First/Last duplicates removed: 150
970 records remaining


In [15]:
#Duplicate emails
for x, y in zip(df['E-Mail'].value_counts(), df['E-Mail'].value_counts().index):
    if x>1:
        print(f'{y} '+str(x))

ceh_0769@yahoo.com 3
patdillard13@gmail.com 3
allanka@miamioh.edu 3
newdlcntr@aol.com 3
itscmh@sbcglobal.net 2
shanae.hespeth@caresource.com 2
lydia.boadaclista@yahoo.com 2
bartlettbl@udayton.edu 2
mbadgley831@att.net 2
taristi6@gmail.com 2
tanishahead1230@yahoo.com 2
aanderson@udayton.edu 2
shakurahmad@hotmail.com 2
ehess@cppsadmin.org 2
dionrgreen@gmail.com 2
carloshill198@gmail.com 2
feliciafg1983@gmail.com 2
James@PACEmentoring.org 2
shirtle@sbcglobal.net 2
brielhope@gmail.com 2
kerrixs5@gmail.com 2
jjames@uuma.org 2
beelermr@mail.uc.edu 2
sue@pretrial.org 2
jennifer@urbanvillagechurch.org 2
diebelalice@gmail.com 2
jagrace@wowway.com 2
Tycurington@aol.com 2
Asiarosegibbs@gmail.com 2
rcartergreen@sbcglobal.net 2
geej@mcohio.org 2


In [16]:
# which organizers have the duplicate emails?
test = df['E-Mail'].value_counts().to_frame().rename({'E-Mail':'count'}, axis=1)
test['E-Mail'] = test.index

test = pd.merge(left=test, right=df, on='E-Mail', how='left')

test[['count', 'E-Mail', 'First Name', 'Last Name', 'Organizer']]

,count,E-Mail,First Name,Last Name,Organizer
0,3,ceh_0769@yahoo.com,Nicole,Bumpus,Edwin
1,3,ceh_0769@yahoo.com,Michael,Colbert,Edwin
2,3,ceh_0769@yahoo.com,Candace,Edwards,Edwin
3,3,patdillard13@gmail.com,Samantha,Bowden,Edwin
4,3,patdillard13@gmail.com,Rebecca,Cochran,Edwin
...,...,...,...,...,...
863,1,gwjones529@gmail.com,Guy,Jones,Edwin
864,1,ofuapolicebrutality@gmail.com,Sabrina,Jordan,Edwin
865,1,knithobbit@gmail.com,Lori,Keith,Edwin
866,1,Laurelkerr1549@gmail.com,Laurel,Kerr,Edwin


# Institution - Not going into EA yet

In [17]:
#Exported csv of all institutions. Bolded ones need to be looked at. Can they be combined? Should some be removed?
# pd.DataFrame(df.loc[:,'Institution'].value_counts().index.sort_values()).to_csv('Institutions.csv')

# View the data

In [18]:
# Look at the data, should probably go after duplicates

df.head()

# df.describe(include='all')

# import pandas_profiling
# profile = df.profile_report(title="Pandas Profiling Report", progress_bar=True)
# profile

,First Name,Last Name,E-Mail,Phone,Don't Add to Hustle,Zip Code,Institution,Leadership Status,Core Team,Relational Organizing Leader,Recruited By,Notes & Reflections,Organizer,Title,Organization,Membership
0,Tayjua,Hines,jhines30@kent.edu,NaN,False,nan,Kent State,1-Prospect,No,No,Me,"Current President of BUS, former member of the...",Akii,NaN,OSA,OSA: Columbus
1,Maggie,Ash,maggie@ignitenational.org,33070474580,True,nan,OSU,NaN,No,No,Me,"Recent graduate from OSU, currently a fellow w...",Akii,NaN,OSA,OSA: Columbus
2,Kelsey,Lowman,Kelsey@ignitenational.org,NaN,False,nan,OSU,1-Prospect,No,No,Me,"Third year at OSU, heavily involved with USG a...",Akii,NaN,OSA,OSA: Columbus
3,Raga,Maddela,maddela.3@buckeyemail.osu.edu,NaN,False,nan,OSU,NaN,No,No,Akii,Graduate student at OSU that was interested in...,Akii,NaN,OSA,OSA: Columbus
4,Johnny,Zhang,zhang.10080@buckeyemail.osu.edu,NaN,False,nan,OSU,NaN,No,No,NaN,NaN,Akii,NaN,OSA,OSA: Columbus


# Counts

In [19]:
#Overall Column counts
df.describe(include='all').loc[['count', 'unique', 'top', 'freq'], :]

,First Name,Last Name,E-Mail,Phone,Don't Add to Hustle,Zip Code,Institution,Leadership Status,Core Team,Relational Organizing Leader,Recruited By,Notes & Reflections,Organizer,Title,Organization,Membership
count,943,932,868,389,955,970,598,785,970,970,541,270,970,29,970,946
unique,685,740,833,374,2,106,255,4,2,2,108,117,9,3,4,8
top,Sarah,Smith,ceh_0769@yahoo.com,93735077630,False,nan,Cleveland State,1-Prospect,No,No,Daniel,ldrshp smmt,Edwin,Pastor,OSA,BFO: Dayton
freq,8,15,3,3,830,648,49,380,907,895,115,73,364,25,376,364


In [20]:
#Column counts by organizer, in order of how many first names they have
df.groupby('Organizer').count().sort_values('First Name', ascending=False)

,First Name,Last Name,E-Mail,Phone,Don't Add to Hustle,Zip Code,Institution,Leadership Status,Core Team,Relational Organizing Leader,Recruited By,Notes & Reflections,Title,Organization,Membership
Organizer,,,,,,,,,,,,,,,
Edwin,364,364,361,242,364,364,189,262,364,364,215,149,0,364,364
Rachael,267,267,233,49,260,267,252,262,267,267,142,31,0,267,267
Daniel,98,94,121,7,124,124,8,123,124,124,123,7,0,124,124
Tami,54,54,47,33,50,54,25,20,54,54,1,20,0,54,54
Kalesha,43,38,37,32,40,43,43,43,43,43,38,29,0,43,43
Akii,42,41,35,3,42,42,21,10,42,42,18,13,0,42,42
Derrick,32,32,0,0,32,32,32,25,32,32,0,17,29,32,32
Rachael1,24,24,24,18,24,24,24,24,24,24,0,0,0,24,0
Janiah,19,18,10,5,19,20,4,16,20,20,4,4,0,20,20


In [21]:
#Cincinnati counts
print('Cincinnati counts: \n')
print(df.loc[df['Membership'].str.contains('Cincinnati', na=False), 'Membership'].value_counts())

print('\n\nTotal: ')
print(df.loc[df['Membership'].str.contains('Cincinnati', na=False), 'Membership'].value_counts().sum())

Cincinnati counts: 

AMOS: Cincinnati    124
Name: Membership, dtype: int64


Total: 
124


In [22]:
#Cleveland counts
print('Cleveland counts: \n')
print(df.loc[df['Membership'].str.contains('Cleveland', na=False), 'Membership'].value_counts())

print('\n\nTotal: ')
print(df.loc[df['Membership'].str.contains('Cleveland', na=False), 'Membership'].value_counts().sum())

Cleveland counts: 

OSA: Cleveland    267
Name: Membership, dtype: int64


Total: 
267


In [23]:
#Columbus counts
print('Columbus counts: \n')
print(df.loc[df['Membership'].str.contains('Columbus', na=False), 'Membership'].value_counts())

print('\n\nTotal: ')
print(df.loc[df['Membership'].str.contains('Columbus', na=False), 'Membership'].value_counts().sum())

Columbus counts: 

OSA: Columbus     42
AMOS: Columbus    32
Name: Membership, dtype: int64


Total: 
74


In [24]:
#Columbus|Cincinnati|Cleveland|Dayton counts
print('Columbus counts: \n')
print(df.loc[df['Membership'].str.contains('Columbus|Cincinnati|Cleveland|Dayton', na=False), 'Membership'].value_counts())

print('\n\nTotal: ')
print(df.loc[df['Membership'].str.contains('Columbus|Cincinnati|Cleveland|Dayton', na=False), 'Membership'].value_counts().sum())

Columbus counts: 

BFO: Dayton         364
OSA: Cleveland      267
AMOS: Cincinnati    124
OSA: Dayton          43
OSA: Columbus        42
AMOS: Columbus       32
Name: Membership, dtype: int64


Total: 
872


In [25]:
#Column counts by "recruited by", in order of how many names they have
df.groupby('Recruited By').count().sort_values('First Name', ascending=False)

,First Name,Last Name,E-Mail,Phone,Don't Add to Hustle,Zip Code,Institution,Leadership Status,Core Team,Relational Organizing Leader,Notes & Reflections,Organizer,Title,Organization,Membership
Recruited By,,,,,,,,,,,,,,,
Daniel,89,85,113,0,115,115,1,115,115,115,0,115,0,115,115
MCJC L&L,73,73,73,20,73,73,42,73,73,73,3,73,0,73,73
Edwin Fuller,42,42,42,37,42,42,26,42,42,42,11,42,0,42,42
Rachael,37,37,35,12,36,37,31,37,37,37,2,37,0,37,37
Rachael,34,34,33,20,33,34,34,34,34,34,4,34,0,34,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Facebook,1,1,1,1,1,1,0,1,1,1,0,1,0,1,1
Evan O'Reilly,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1
Evan,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1


In [26]:
#Column counts by "Leadership Status", in order of how many names they have
df.groupby('Leadership Status').count().sort_values('First Name', ascending=False)

,First Name,Last Name,E-Mail,Phone,Don't Add to Hustle,Zip Code,Institution,Core Team,Relational Organizing Leader,Recruited By,Notes & Reflections,Organizer,Title,Organization,Membership
Leadership Status,,,,,,,,,,,,,,,
1-Prospect,353,347,344,89,375,380,198,380,380,249,56,380,10,380,366
2-Volunteer,236,234,227,127,231,236,129,236,236,179,53,236,2,236,236
3-Member Leader,139,139,112,35,139,139,129,139,139,81,37,139,10,139,129
4-Super Leader,30,28,24,13,27,30,29,30,30,22,6,30,2,30,30


In [27]:
df['Leadership Status'].value_counts()

1-Prospect         380
2-Volunteer        236
3-Member Leader    139
4-Super Leader      30
Name: Leadership Status, dtype: int64

In [28]:
df['Leadership Status'].value_counts().sum()

785

# Export Leaders and Duplicates

In [29]:
# Output your df as an Excel document
# pd_writer = pd.ExcelWriter('OOC 2021 Leaders Cleaned.xlsx')
# df.to_excel(pd_writer, index=False, index_label=None, sheet_name="Master Sheet", encoding='UTF-8')
# pd_writer.save()
# pd_writer.close()

#Output your df as a csv
df.to_csv('OOC 2022 Leaders Cleaned.csv', index=False)

In [30]:
#Export duplicates, have organizers combine them as best they can and dedup from there?
# pd_writer = pd.ExcelWriter('OOC 2021 Leaders Duplicates.xlsx')
# duplicates.to_excel(pd_writer, index=False, index_label=None, encoding='UTF-8')
# pd_writer.save()
# pd_writer.close()

# Export duplicates as csv
# duplicates.to_csv('OOC 2022 Leaders Duplicates.csv', index=False)